<a href="https://colab.research.google.com/github/AdamClarkStandke/TinyMachineLearning/blob/main/ClassificationTFLuNano.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 2.x

In [1]:
import numpy as np
import pathlib
import tensorflow as tf
import tensorflow_datasets as tfds
import zipfile
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.models import Model

In [2]:
MODEL_ALPHA = 0.35
MODEL_INPUT_WIDTH = 48
MODEL_INPUT_HEIGHT = 48
TFL_MODEL_FILE = "recognize.tflite"
TFL_MODEL_HEADER_FILE = "recognize_model.h"
TF_MODEL = "drive/MyDrive/recognize/"

In [ ]:
with zipfile.ZipFile("dataset.zip", 'r') as zip_ref:
    zip_ref.extractall(".")
data_dir = "dataset"

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  interpolation="bilinear",
  image_size=(MODEL_INPUT_WIDTH, MODEL_INPUT_HEIGHT)
  )


val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  interpolation="bilinear",
  image_size=(MODEL_INPUT_WIDTH, MODEL_INPUT_HEIGHT)
  )

Found 40 files belonging to 2 classes.
Using 32 files for training.
Found 40 files belonging to 2 classes.
Using 8 files for validation.


In [ ]:
class_names = train_ds.class_names
num_classes = len(class_names)
print(class_names)

['bathroom', 'kitchen']


In [ ]:
rescale = tf.keras.layers.Rescaling(1./255, offset= -1)
train_ds = train_ds.map(lambda x, y: (rescale(x), y))
val_ds   = val_ds.map(lambda x, y: (rescale(x), y))

In [ ]:
base_model = MobileNetV2(input_shape=(MODEL_INPUT_WIDTH, MODEL_INPUT_HEIGHT, 3),
                         include_top=False,
                         weights='imagenet',
                         alpha=0.35)
base_model.trainable = False

feat_extr = base_model

In [ ]:
augmen = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
])

train_ds = train_ds.map(lambda x, y: (augmen(x), y))
val_ds   = val_ds.map(lambda x, y: (augmen(x), y))

In [ ]:
global_avg_layer = tf.keras.layers.GlobalAveragePooling2D()
dense_layer = tf.keras.layers.Dense(num_classes, activation='softmax')

In [ ]:
inputs = tf.keras.Input(shape=(MODEL_INPUT_WIDTH, MODEL_INPUT_HEIGHT, 3))
x = global_avg_layer(feat_extr.layers[-1].output)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = dense_layer(x)
model = tf.keras.Model(inputs=feat_extr.inputs, outputs=outputs)

In [ ]:
lr = 0.0005
model.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=False),
  metrics=['accuracy'])
     

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 48, 48, 3)]  0           []                               
                                                                                                  
 Conv1 (Conv2D)                 (None, 24, 24, 16)   432         ['input_1[0][0]']                
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 24, 24, 16)   64          ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 24, 24, 16)   0           ['bn_Conv1[0][0]']               
                                                                                              

In [ ]:
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=100
)

In [ ]:
model.save(TF_MODEL)

# KEY: Quantize the TensorFlow model with the TFLite converter!!!

In [8]:
# Key: TFLite Converter settings for converting TensorFlow model to the TFLite
# FlatBuffer format (i.e. contigious array of bytes, such as 0x1f form) which 
# contains the model arhitecture and the weights of the traininable layers 
# (i.e. in this case the 2,562 weights from the dense layer)
converter = tf.lite.TFLiteConverter.from_saved_model(TF_MODEL)
converter.representative_dataset = tf.lite.RepresentativeDataset(representative_data_gen)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
tfl_model = converter.convert()

In [9]:
size_tfl_model = len(tfl_model)
print(len(tfl_model), "bytes")

617384 bytes


In [ ]:
#Key: use xxd to convert the flatbuffer into a C header file to be imported by Arduino IDE
# i.e. converts it from TFlite's flatbuffer into C-byte array for microcontroller
# efficiency
open("model.tflite", "wb").write(tfl_model)
!apt-get update && apt-get -qq install xxd
!xxd -c 60 -i model.tflite > indoor_scene_recognition.h

## Testing TFlite model (quantizied model) on the test data 

In [3]:
with zipfile.ZipFile("test_samples.zip", 'r') as zip_ref:
    zip_ref.extractall(".")
test_dir = "test_dataset"

In [35]:
def representative_data_gen():
  for i_value, o_value in repr_ds.batch(1).take(48):
    yield [i_value]

test_ds = tf.keras.utils.image_dataset_from_directory(test_dir,
                                                      interpolation="bilinear",
                                                      image_size=(MODEL_INPUT_WIDTH, MODEL_INPUT_HEIGHT))
rescale = tf.keras.layers.Rescaling(1./255, offset= -1)
test_ds  = test_ds.map(lambda x, y: (rescale(x), y))

Found 10 files belonging to 2 classes.


In [36]:
# Initialize the TFLite interpreter
interpreter = tf.lite.Interpreter(model_content=tfl_model)

# Allocate the tensors
interpreter.allocate_tensors()

In [37]:
# input details of the test data (i.e. un-quantizied images/tensors)
i_details = interpreter.get_input_details()[0]
# output details of the model (i.e. the classification)
o_details = interpreter.get_output_details()[0]
# the quantizied information that will be used to map the input values from float domain to int domain
i_quant = i_details['quantization_parameters']
i_scale = i_quant['scales'][0] # i.e. mapping 
i_zero_point = i_quant['zero_points'][0] # i.e. offset 

In [38]:
test_ds0 = test_ds.unbatch()

num_correct_samples = 0
num_total_samples   = len(list(test_ds0.batch(1)))

for i_value, o_value in test_ds0.batch(1):
  i_value = (i_value / i_scale) + i_zero_point
  i_value = tf.cast(i_value, dtype=tf.int8)
  interpreter.set_tensor(i_details["index"], i_value)
  interpreter.invoke()
  o_pred = interpreter.get_tensor(o_details["index"])[0]

  if np.argmax(o_pred) == o_value:
    num_correct_samples += 1

print("Accuracy:", num_correct_samples/num_total_samples)

Accuracy: 0.4


In [23]:
# open("model.tflite", "wb").write(tfl_model)
# !apt-get update && apt-get -qq install xxd
# !xxd -c 60 -i model.tflite > indoor_scene_recognition.h